#### LangChain Prompts模板的输入工程
使用prompts模板实现更加高级和灵活的提示词工程

优秀的提示词的特点：
- 【立角色】：引导AI进入具体场景，赋予其行家身份
- 【述问题】：告诉AI你的困惑和问题，以及背景信息
- 【定目标】：告诉AI你的需求，希望达成的目标
- 【补要求】：告诉AI回答时注意什么，或者如何回复

提示词模板：
1. 将提示词提炼成模板的形式
2. 实现提示词的复用、版本管理、动态变化等
  

##### PromptTemplate 字符串模板
LangChain支持ChatModels和LLMModels，可以使用字符串模板来生成对话。
提供了PromptTemplate类来处理字符串模板。
提供了ChatPromptTemplate类来处理对话模板。

In [ ]:
# 字符串模板
from langchain_core.prompts import PromptTemplate

# 创建字符串模板
prompt = PromptTemplate.from_template("你是一个{name}，帮我起1个具有{county}特色的{sex}名字。")
prompt.format(name="算命大师", county="法国", sex="女孩")

##### ChatPromptTemplate 对话模板
可以指定消息的角色，例如 "system"（系统）、"human"（人类）或 "ai"（AI）
这使得你可以更精确地控制聊天对话的上下文和行为

In [ ]:
# 对话模板具有结构，ChatModels
from langchain_core.prompts import ChatPromptTemplate

# 有不同的角色，系统、人类、AI
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师，你的名字叫{name}"),
        ("human", "你好{name}，你感觉如何？"),
        ("ai", "你好！我的状态非常好！"),
        ("human", "你叫什么名字呢？"),
        ("ai", "你好！我叫{name}。"),
        ("human", "{user_input}"),
    ]
)

# 参数传递给模板
chat_template.format_messages(name="陈大师", user_input="你的爸爸是谁呢？")

In [ ]:
# 直接引入SystemMessage、HumanMessage、AIMessage消息类创建消息
# 与上方是等价的，但是编写方式更加灵活
# 它表示一个具体的、已生成的消息。它不涉及提示模板或变量替换。
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

system = SystemMessage(
    content="你是一个起名大师",
    # 附加参数
    additional_kwargs={"大师姓名": "陈瞎子"}
)

human = HumanMessage(
    content="请问大师叫什么？"
)

ai = AIMessage(
    content="我叫陈瞎子"
)

[system, human, ai]

##### ChatMessagePromptTemplate 聊天消息提示模板
允许你指定任意角色名称，适用于支持自定义角色的聊天模型。
这使得它比 HumanMessagePromptTemplate、AIMessagePromptTemplate 和 SystemMessagePromptTemplate 等固定角色更通用

In [ ]:
from langchain_core.prompts import AIMessagePromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatMessagePromptTemplate

# 定义模板
prompt = "愿{subject}与你同在！"

# AIMessagePromptTemplate 默认的AI角色
# SystemMessagePromptTemplate 默认的system角色
# HumanMessagePromptTemplate 默认的human角色

# 自定义角色模板，指定模板。使用role来设定对话角色
chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者", template=prompt)
# 动态参数
chat_message_prompt.format(subject="原力")

##### 自定义模板

In [ ]:
# 实现函数大师应用：可以根据函数名称，去查找函数代码，并给出中文的代码说明

from langchain_core.prompts import StringPromptTemplate

# 定义一个简单的函数作为示例效果
def hello_world():
    """
    一个简单的hello world函数
    """
    print("hello world!")
    return "hello world!"

PROMPT = """\
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称：{function_name}
源代码：
{source_code}
代码解释：
"""

# 引入通过函数名获取源代码的工具包
import inspect

def get_source_code(function_name):
    """
    获取函数的源代码
    """
    # 获取函数对象
    function = globals().get(function_name)
    # 获取函数源代码
    source_code = inspect.getsource(function)
    return source_code

# 自定义的Prompt模板，继承StringPromptTemplate
class CustPrompt(StringPromptTemplate):
    """
    自定义的Prompt模板
    """
    def format(self, **kwargs):
        """
        格式化函数
        """
        # 获取函数名称
        function_name = kwargs.get("function_name")
        # 获取函数源代码
        source_code = get_source_code(function_name)
        # 生成提示词模板
        prompt = PROMPT.format(
            function_name=function_name, 
            source_code=source_code
        )
        return prompt

# 使用自定义模板时，首先创建 CustPrompt 类的实例，并传入一个包含函数名称的字典作为 input_variables。
cust_prompt = CustPrompt(input_variables=["function_name"])
# 然后，调用 format 方法，并传入实际的函数对象 hello_world，生成最终的提示词模板。
pm = cust_prompt.format(function_name=hello_world.__name__)

print(pm)

# 和LLM连接起来，让其解释代码
from langchain_openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(".env")

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

llm = OpenAI(
    openai_api_base=api_base,
    openai_api_key=api_key,
    model=model_name,
    temperature=0,
)

msg = llm.invoke(pm)
print(msg)

##### 使用F-string与jinja2来实现提示词模板格式化

In [ ]:
# f-string是python内置的一种模板引擎
from langchain_core.prompts import PromptTemplate

# f-string模板使用
f_string_template = """
给我讲一个关于{name}的{what}故事。
"""

prompt = PromptTemplate.from_template(f_string_template)
prompt.format(name="翠花", what="悲伤")

###### 安装jinja2依赖

In [ ]:
!pip install jinja2

In [ ]:
# Jinja2是一个灵活、搞笑的Python模板引擎，可以方便的生成各种标记格式的文档。
from langchain_core.prompts import PromptTemplate

# jinja2模板使用
jinja2_template = "给我将一个关于{{name}}的{{what}}故事。"

# 创建PromptTemplate实例，传入模板并且指定模板类型为jinja2
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")
prompt.format(name="翠花", what="悲伤")

##### 组合式提示词模板
+ Final prompt：最终返回的提示词模板
+ Pipeline prompts：组成提示词管道的模板

In [ ]:
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts import PromptTemplate

##### 组合模板：三层提示词设计

In [ ]:
# 三层提示词示例(最终生成的效果)
PROMPT = """
你是一个非常开朗的男孩，你是中国人，住在一个非常美丽的城市。
你总是穿蓝色衣服，戴着绿色手表。
你从不说自己是一个人工智能。
"""

# Final Prompt 由一些列变量组成
full_template = """{character}
{behavior}
{prohibit}"""

full_prompt = PromptTemplate.from_template(full_template)

+ 第一层基本性格设计

In [ ]:
character_template = """你是{person}，你有着{xingge}."""
character_prompt = PromptTemplate.from_template(character_template)

+ 第二层行为设计

In [ ]:
behavior_template = """你遵从以下的行为：
{behavior_list}
"""
behavior_prompt = PromptTemplate.from_template(behavior_template)

+ 第三层不允许的行为

In [ ]:
prohibit_template = """你不允许有以下行为：
{prohibit_list}
"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

+ 将三层提示词组合起来

In [ ]:
input_prompts = [
    ("character", character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt)
]

# 创建PipelinePromptTemplate实例，将多个提示模板连接在一起
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

+ 打印下组合后模板里的可传递参数

In [ ]:
pipeline_prompt.input_variables

+ 调整组合后的提示词模板（传递参数）

In [ ]:
# PipelinePromptTemplate实例格式化提示
pm = pipeline_prompt.format(
    person="埃隆马斯克",
    xingge="钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是移民火星。",
    behavior_list="1.你喜欢冒险 \n 2.你非常崇拜爱因斯坦 \n 3.你总是乐于助人，回答用户各种问题。",
    prohibit_list="1.你不可以说自己是一个人工智能或者机器人 \n 2.当你不知道答案的时候，可以说让我再想想。"
)

print(pm)

##### 序列化：使用文件来管理提示词模板
+ 将提示词模板以文件形式保存
+ 便于共享
+ 便于版本管理
+ 便于存储
+ 支持常见格式 （json/yaml/txt）

In [ ]:
# 引入包
from langchain_core.prompts import load_prompt

# 加载提示词模板文件

# 加载yaml格式的prompt模板文件
prompt = load_prompt(path="simple_prompt.yaml", encoding="utf-8")
# 格式化提示词
print(prompt.format(name="小黑", what="悲伤"))

# 加载json格式的prompt模板文件
prompt2 = load_prompt(path="simple_prompt.json", encoding="utf-8")
# 格式化提示词
print(prompt2.format(name="小红", what="搞笑"))

In [ ]:
# 支持加载文件格式的模板,并且对prompt的最终解析结果进行自定义格式化
from langchain.output_parsers import RegexParser

# 定义 Prompt 并包含定义解析方式
prompt = PromptTemplate(
    input_variables=["question", "student_answer"],
    template="Given the following question and student answer, provide a correct answer and score the student answer.\nQuestion: {question}\nStudent Answer: {student_answer}\nCorrect Answer:",
    output_parser=RegexParser(
        regex="(.*?)\\nScore: (.*)",  # 正则匹配
        output_keys=["answer", "score"],  # 输出的key
    )
)

# 解析输出
parsed_output = prompt.output_parser.parse("George Washington was born in 1732 and died in 1799.\nScore: 1/2")
print(parsed_output)

In [ ]:
# 使用提示词模板文件实现与上面相同功能
import json
from langchain_core.prompts import load_prompt

# 该方式目前版本不支持这样使用 已过时
# 加载json格式的prompt模板文件
# prompt = load_prompt(path="prompt_with_output_parser.json", encoding="utf-8")
# parsed_output = prompt.output_parser.parse(
#     "George Washington was born in 1732 and died in 1799.\nScore: 1/2"
# )
# print(parsed_output)

# 读取 JSON 文件内容
with open("prompt_with_output_parser.json", "r", encoding="utf-8") as f:
    prompt_data = json.load(f)

# 创建正则解析器 (RegexParser)
output_parser_config = prompt_data["output_parser"]
output_parser = RegexParser(
    regex=output_parser_config["regex"],
    output_keys=output_parser_config["output_keys"]
)

# 创建提示词模板 (PromptTemplate)
prompt_template = PromptTemplate(
    input_variables=prompt_data["input_variables"],
    template=prompt_data["template"]
)

# 提供输入数据
question = "What is 2+2?"
student_answer = "4"

# 格式化提示并运行
formatted_prompt = prompt_template.format(question=question, student_answer=student_answer)
print("Formatted Prompt:", formatted_prompt)

# 模拟 LLM 输出
llm_output = "George Washington was born in 1732 and died in 1799.\nScore: 1/2"

# 使用 output_parser 解析 LLM 输出
parsed_output = output_parser.parse(llm_output)
print("Parsed Output:", parsed_output)